In [3]:
# !pip install opencv-python pillow


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\kevin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import cv2
import os
from PIL import Image
import numpy as np
import shutil

In [3]:

def flip_image_and_labels(image_path, label_path, flip_mode, output_image_path, output_label_path):
    # 이미지 불러오기
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # 이미지 뒤집기
    flipped_image = cv2.flip(image, flip_mode)

    # 라벨 파일 불러오기 및 뒤집기
    flipped_labels = []
    with open(label_path, 'r') as file:
        for line in file:
            class_id, x_center, y_center, bbox_width, bbox_height = map(float, line.split())
            class_id = int(class_id)
            if flip_mode == 1:  # 좌우 뒤집기
                x_center = 1 - x_center
            elif flip_mode == 0:  # 상하 뒤집기
                y_center = 1 - y_center
            flipped_labels.append((class_id, x_center, y_center, bbox_width, bbox_height))

    # 결과 저장
    cv2.imwrite(output_image_path, flipped_image)
    with open(output_label_path, 'w') as file:
        for label in flipped_labels:
            file.write(' '.join(map(str, label)) + '\n')

def adjust_brightness_and_save(image_path, label_path, brightness_factor, output_image_path, output_label_path):
    # 이미지 불러오기
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 밝기 조정
    adjusted_image = np.clip(image * brightness_factor, 0, 255).astype(np.uint8)

    # 이미지 및 라벨 저장
    cv2.imwrite(output_image_path, cv2.cvtColor(adjusted_image, cv2.COLOR_RGB2BGR))

    # 라벨 파일 복사
    shutil.copy(label_path, output_label_path)

def apply_blur_and_save(image_path, label_path, blur_strength, output_image_path, output_label_path):
    # 이미지 불러오기
    image = cv2.imread(image_path)

    # 블러 효과 적용
    blurred_image = cv2.GaussianBlur(image, (blur_strength, blur_strength), 0)

    # 이미지 및 라벨 저장
    cv2.imwrite(output_image_path, blurred_image)
    shutil.copy(label_path, output_label_path)

def rotate_image_and_labels(image_path, label_path, output_image_path_left, output_image_path_right, output_label_path_left, output_label_path_right):
    # 이미지 불러오기
    image = cv2.imread(image_path)

    # 왼쪽, 오른쪽으로 90도 회전
    rotated_left_image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    rotated_right_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)

    # 회전된 이미지 저장
    cv2.imwrite(output_image_path_left, rotated_left_image)
    cv2.imwrite(output_image_path_right, rotated_right_image)

    # 라벨 파일 불러오기
    with open(label_path, 'r') as file:
        labels = file.readlines()

    # 라벨 수정 및 저장
    for idx, rotation in enumerate([(output_label_path_left, rotated_left_image, 0), (output_label_path_right, rotated_right_image, 1)]):
        output_label_path, rotated_image, rot_index = rotation
        h, w, _ = rotated_image.shape
        with open(output_label_path, 'w') as file:
            for label in labels:
                class_id, x_center, y_center, bbox_width, bbox_height = map(float, label.split())
                class_id = int(class_id)
                new_x_center = y_center
                new_y_center = 1 - x_center
                new_width = bbox_height
                new_height = bbox_width
                if rot_index == 1:  # 오른쪽으로 회전한 경우
                    new_x_center = 1 - y_center
                    new_y_center = x_center
                file.write(f"{class_id} {new_x_center} {new_y_center} {new_width} {new_height}\n")

In [7]:
image_dir = './train/images/'
label_dir = './train/labels/'
output_dir = './aug_data/'

# 이미지와 라벨 폴더 생성
output_image_dir = os.path.join(output_dir, "images")
output_label_dir = os.path.join(output_dir, "labels")
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

brightness_factors = [0.6, 1.4]  # 밝기 조절 요소 (0.6: 어둡게, 1.4: 밝게)
blur_strengths = [5, 9]  # 블러 강도(클수록 흐릿)
angles = [-90, 90] 

for filename in os.listdir(image_dir):
    if filename.endswith(".jpg"):
        base_filename = os.path.splitext(filename)[0]

        # 원본 이미지 및 라벨 경로
        image_path = os.path.join(image_dir, filename)
        label_path = os.path.join(label_dir, base_filename + ".txt")

        # 좌우 뒤집기
        flip_image_and_labels(
            image_path, label_path, 1, 
            os.path.join(output_image_dir, base_filename + "_flip_horizontal.jpg"),
            os.path.join(output_label_dir, base_filename + "_flip_horizontal.txt")
        )

        # 상하 뒤집기
        flip_image_and_labels(
            image_path, label_path, 0, 
            os.path.join(output_image_dir, base_filename + "_flip_vertical.jpg"),
            os.path.join(output_label_dir, base_filename + "_flip_vertical.txt")
        )

        # 밝기 조정
        for factor in brightness_factors:
            # 밝기 조정 및 저장
            adjusted_image_path = os.path.join(output_image_dir, f"{base_filename}_brightness_{factor}.jpg")
            adjusted_label_path = os.path.join(output_label_dir, f"{base_filename}_brightness_{factor}.txt")
            adjust_brightness_and_save(
                image_path, label_path, factor, adjusted_image_path, adjusted_label_path
            )

        # 블러 적용
        for blur_strength in blur_strengths:
            # 블러 적용 및 저장
            apply_blur_and_save(
                image_path, label_path, blur_strength,
                os.path.join(output_image_dir, f"{base_filename}_blur_{blur_strength}.jpg"),
                os.path.join(output_label_dir, f"{base_filename}_blur_{blur_strength}.txt")
            )
        
        # 90도 회전
        rotate_image_and_labels(
            image_path, label_path, 
            os.path.join(output_image_dir, base_filename + "_rotate_left_90.jpg"),
            os.path.join(output_image_dir, base_filename + "_rotate_right_90.jpg"),
            os.path.join(output_label_dir, base_filename + "_rotate_left_90.txt"),
            os.path.join(output_label_dir, base_filename + "_rotate_right_90.txt")
        )